# Coherence-based QPP Predictors for Dense Retrieval

##### TCT-ColBERT full process

In this notebook, we demonstrate how we obtain our results for the QPP Dense Coherence-based predictors, namely pairRatio and A-pairRatio. We have obtained the retrieval results files for the TCT-ColBERT retrieval method, as seen below. However, the reader can produce their own results files and replace the corresponding csv files in the arguments.

First, install pyterrier_dr from https://github.com/terrierteam/pyterrier_dr/tree/master. 

In [ ]:
%pip install --force-reinstall --no-deps git+https://github.com/terrierteam/pyterrier_dr.git@docids

In [ ]:
# %pip install 'numpy<2'

In [ ]:
%pip install -q sentence_transformers

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Make sure you have installed the latest version of pyterrier.

In [2]:
#%pip install --upgrade git+https://github.com/terrier-org/pyterrier.git
import pyterrier as pt
pt.init()

/opt/miniconda3/envs/genir/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [ ]:
#pt.__version__

In [ ]:
#%conda install -c pytorch faiss-gpu

In [5]:
import pyterrier_dr

Here, we define the retrieval model using the corresponding index for TCT (please check that you have put the right index before proceeding).

In [6]:
model = pyterrier_dr.TctColBert('castorini/tct_colbert-v2-hnp-msmarco')
index = pyterrier_dr.NumpyIndex("/nfs/global_indices/msmarco-passage.tct-hnp", docids=True)
retr_pipeline = model >> index

/opt/miniconda3/envs/genir/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [10]:
index.docids

True

In [11]:
def get_embs(row):
    return index.docnos_and_data()[1][row.docid]

Install required libraries.

In [12]:
import pandas as pd
import numpy as np
import torch
from scipy import stats
from scipy.stats import spearmanr,kendalltau
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance_matrix
from math import sqrt, log
from pyterrier.measures import *
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

## Get Exp_Results

We get the experimental results with all three evaluation metrics for the TREC DL datasets.

In [16]:
dataset = pt.get_dataset('trec-deep-learning-passages')

In [34]:
per_query_results = pt.Experiment(
    [retr_pipeline],
    dataset.get_topics("test-2019"),
    dataset.get_qrels("test-2019"),
    [MAP(rel=2)@100, nDCG@10, RR(rel=2)@10],
    perquery=True, filter_by_qrels=True,
)

In [47]:
per_query_results.head()

,name,qid,measure,value
27,Compose(<pyterrier_dr.tctcolbert_model.TctColB...,1037798,nDCG@10,0.370031
28,Compose(<pyterrier_dr.tctcolbert_model.TctColB...,1037798,AP(rel=2)@100,0.178571
29,Compose(<pyterrier_dr.tctcolbert_model.TctColB...,1037798,RR(rel=2)@10,1.000000
63,Compose(<pyterrier_dr.tctcolbert_model.TctColB...,104861,nDCG@10,1.000000
64,Compose(<pyterrier_dr.tctcolbert_model.TctColB...,104861,AP(rel=2)@100,0.514095


In [48]:
per_query_results_2020 = pt.Experiment(
    [retr_pipeline],
    dataset.get_topics("test-2020"),
    dataset.get_qrels("test-2020"),
    [MAP(rel=2)@100, nDCG@10, RR(rel=2)@10],
    perquery=True, filter_by_qrels=True,
)

## Get Test Topics

Get query sets as follows:

In [49]:
test_topics = dataset.get_topics("test-2019").merge(dataset.get_qrels("test-2019")[['qid']].drop_duplicates())
test_topics.head()

,qid,query
0,156493,do goldfish grow
1,1110199,what is wifi vs bluetooth
2,1063750,why did the us volunterilay enter ww1
3,130510,definition declaratory judgment
4,489204,right pelvic pain causes


In [50]:
test_topics_2020 = dataset.get_topics("test-2020").merge(dataset.get_qrels("test-2020")[['qid']].drop_duplicates())

Define our Retrieval Pipeline

In [25]:
new_pipe = retr_pipeline >> pt.apply.doc_embs(get_embs) >> pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')

This line adds the embedding vectors to the query sets.

In [45]:
test_queries = model.transform(test_topics) ###get query embs
test_queries_2020 = model.transform(test_topics_2020)

### Get retrieved results using our pipeline.

In [52]:
new_pipe_100 = (retr_pipeline % 100) >> pt.apply.doc_embs(get_embs) >> pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')
all_res_embs_100 = new_pipe_100.transform(test_topics)
combined_100 = test_queries.merge(all_res_embs_100)

This is how the final retrieved results look like: Doc_embs are the document embedded representations of the per query retrieved documents, while query_vec is the embedded representation of each query.

In [53]:
combined_100.head()

,qid,query,query_vec,docno,score,docid,rank,doc_embs,text
0,156493,do goldfish grow,"[0.09275106, 0.31872433, 0.20686068, -0.016680...",2928707,81.193695,2928707,0,"[0.100064516, 0.19567434, 0.30560207, -0.01379...",Goldfish Only Grow to the Size of Their Enclos...
1,156493,do goldfish grow,"[0.09275106, 0.31872433, 0.20686068, -0.016680...",1960257,81.051804,1960257,1,"[0.08526538, 0.21067119, 0.3275369, -0.0055650...",Goldfish Only Grow to the Size of Their Enclos...
2,156493,do goldfish grow,"[0.09275106, 0.31872433, 0.20686068, -0.016680...",1960255,81.024101,1960255,2,"[0.07921311, 0.20023422, 0.24291284, 0.0113921...",Rating Newest Oldest. Best Answer: Goldfish do...
3,156493,do goldfish grow,"[0.09275106, 0.31872433, 0.20686068, -0.016680...",8182162,80.989685,8182162,3,"[0.050824236, 0.21521352, 0.31407806, -0.07441...","Depending on his type and his environment, gol..."
4,156493,do goldfish grow,"[0.09275106, 0.31872433, 0.20686068, -0.016680...",2612493,80.742928,2612493,4,"[0.09987633, 0.18474764, 0.25994685, -0.015145...","In clean, uncrowded conditions in tanks or pon..."


### Dense Coherence-based QPP predictors: Calculation

Now, we show how we calculate our proposed predictors. First, we define a function for pairRatio.

In [38]:
def pair_ratio(df_embs, lim_1, lim_2, lim_3, measure_x, per_query_res):
    rows = []
    
    for qid, group in pt.tqdm(df_embs.groupby('qid'), unit='q'):
        embs_list = np.vstack(group.doc_embs.tolist())
        W_mat = pd.DataFrame(cosine_similarity(embs_list,dense_output=True))
        mean_top = W_mat.iloc[0:lim_1, 0:lim_1].values.mean()
        mean_bottom = W_mat.iloc[lim_2:lim_3, lim_2:lim_3].values.mean()
        mean_vs = mean_top/mean_bottom
        rows.append([qid,mean_vs])
    df_sim = pd.DataFrame(rows, columns=['qid', 'mean_vs'])
    merged = df_sim.merge(per_query_res, on = 'qid')
    merged = merged[merged.measure==measure_x]
    #corr_pearson = stats.pearsonr(merged['value'], merged['mean_vs'])[0]
    #corr_spearman = spearmanr(merged['value'], merged['mean_vs']).correlation
    corr_kendall = kendalltau(merged['value'], merged['mean_vs'])
    print (corr_kendall)

In the above function, lim_3 corresponds to the rank cutoff of the retrieved results, lim_1 is where the upper matrix stops, and lim_2 is where the lower matrix starts. For measure_x, replace with the metric of interest from the per_query_results, choose AP(rel=2)@100, nDCG@10, or RR(rel=2)@10. 

Test pairRatio: Here we use AP@100 for a cutoff at rank 100. Replace with other metrics to see what happens to NDCG@1- and MRR@10, or cutoffs by adjusting the limits. You can also use the results for TREC DL20 using per_query_results_2020. To get a different correlation metric, simply uncomment the lines for pearson and spearman's correlation.

Here, we demonstrate a test with a rank cutoff of 100. For a top-50 results list, the corresponding lim_1 and lim_2 will be in intervals of 5 from 5 to 35, while lim_3 would be 50.

In [39]:
for lim1 in [10, 20, 30, 40, 50, 60, 70, 80]:
    print("lim1 %d" % lim1)
    for lim2 in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
        print("lim2 %d" % lim2)
        pair_ratio(combined_100, lim1, lim2, 100, 'AP(rel=2)@100', per_query_results)
        print("")

lim1 10
lim2 10


100%|██████████| 43/43 [00:00<00:00, 1030.32q/s]


SignificanceResult(statistic=0.20265780730897007, pvalue=0.055470577997765856)

lim2 20


100%|██████████| 43/43 [00:00<00:00, 1113.58q/s]


SignificanceResult(statistic=0.19379844961240308, pvalue=0.06703361454097385)

lim2 30


...

SignificanceResult(statistic=0.052048726467331115, pvalue=0.6228078793721249)

lim2 80


100%|██████████| 43/43 [00:00<00:00, 1204.66q/s]


SignificanceResult(statistic=0.07641196013289035, pvalue=0.470224382087596)

lim2 90


100%|██████████| 43/43 [00:00<00:00, 1209.67q/s]


SignificanceResult(statistic=0.08305647840531562, pvalue=0.4325083682789408)



In [42]:
def adjusted_pair_ratio(df_embs, lim_1, lim_2, lim_3, measure_x, per_query_res):
    rows = []
    
    for qid, group in pt.tqdm(df_embs.groupby('qid'), unit='q'):
        embs_list = np.vstack(group.doc_embs.tolist())
        query_embs = group.iloc[0].query_vec
        score_list = group.score
        W_mat = cosine_similarity(embs_list,dense_output=True)
        score_exp = np.expand_dims(score_list,axis=1)
        pair_mat = np.dot(score_exp,score_exp.T)
        weighted_mat = W_mat@pair_mat
        W_mat_new = pd.DataFrame(weighted_mat)
        mean_top = W_mat_new.iloc[0:lim_1, 0:lim_1].values.mean()
        mean_bottom = W_mat_new.iloc[lim_2:lim_3, lim_2:lim_3].values.mean()
        mean_vs = mean_top/mean_bottom
        rows.append([qid,mean_vs])   
    df_sim = pd.DataFrame(rows, columns=['qid', 'mean_vs'])
    merged = df_sim.merge(per_query_res, on = 'qid')
    merged = merged[merged.measure==measure_x]
    #corr_person = stats.pearsonr(merged['value'], merged['mean_vs'])[0]
    #corr_spearman = spearmanr(merged['value'], merged['mean_vs']).correlation
    corr_kendall = kendalltau(merged['value'], merged['mean_vs'])
    print(corr_kendall)

Test A-pairRatio in a similar way as pairRatio above.

In [43]:
for lim1 in [10, 20, 30, 40, 50, 60, 70, 80]:
    print("lim1 %d" % lim1)
    for lim2 in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
        print("lim2 %d" % lim2)
        adjusted_pair_ratio(combined_100, lim1, lim2, 100, 'AP(rel=2)@100', per_query_results)
        print("")

lim1 10
lim2 10


100%|██████████| 43/43 [00:00<00:00, 609.07q/s]


SignificanceResult(statistic=0.26688815060908083, pvalue=0.011663453960855275)

lim2 20


100%|██████████| 43/43 [00:00<00:00, 552.16q/s]


SignificanceResult(statistic=0.2757475083056478, pvalue=0.009163596363952935)

lim2 30


...

SignificanceResult(statistic=0.17386489479512734, pvalue=0.1003682399820881)

lim2 80


100%|██████████| 43/43 [00:00<00:00, 853.96q/s]


SignificanceResult(statistic=0.17386489479512734, pvalue=0.1003682399820881)

lim2 90


100%|██████████| 43/43 [00:00<00:00, 860.61q/s]

SignificanceResult(statistic=0.19379844961240308, pvalue=0.06703361454097385)



### Top1(monoT5)

On top of our dense coherence-based predictors, we propose a baseline predictor on the supervised side. Below, we show how we obtain it.

First, install the pyterrier plugin for Mono and Duo T5 from https://github.com/terrierteam/pyterrier_t5.

In [ ]:
pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

In [55]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker

In [56]:
monoT5 = MonoT5ReRanker()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Now, we update the pipeline by getting the document embeddings (first line). Then, in the second line, we define the cross-encoder pipeline for the proposed predictor.

In [57]:
new_pipe = retr_pipeline >> pt.apply.doc_embs(get_embs) >> pt.text.get_text(pt.get_dataset('irds:msmarco-passage'), 'text')
cross_encoder_pipe = new_pipe %1 >> monoT5 >> pt.apply.qpp_monot5(lambda df: df["score"])

We then transform the queries to get the final results, and we merge with the evaluation metrics to get the final correlation.

In [58]:
allres_t5 = cross_encoder_pipe.transform(test_topics)
allres_t5_2020 = cross_encoder_pipe.transform(test_topics_2020)

monoT5: 100%|██████████| 14/14 [00:11<00:00,  1.17batches/s]


In [59]:
merged = allres_t5.merge(per_query_results)
merged = merged[merged.measure=='AP(rel=2)@100']
corr_kendall = kendalltau(merged['value'], merged['qpp_monot5'])
print(corr_kendall)

SignificanceResult(statistic=0.058693244739756366, pvalue=0.5791222556173692)


In [60]:
merged = allres_t5_2020.merge(per_query_results_2020)
merged = merged[merged.measure=='AP(rel=2)@100']
corr_kendall = kendalltau(merged['value'], merged['qpp_monot5'])
#corr_pearson = stats.pearsonr(merged['value'], merged['qpp_monot5'])[0]
#corr_spearman = spearmanr(merged['value'], merged['qpp_monot5']).correlation
print(corr_kendall)

SignificanceResult(statistic=0.27962252669275495, pvalue=0.0028428020282334435)


Simply uncomment the corresponding lines to get the pearson and spearman's correlations.